### The `Selection` Widgets

+ Dropdown
+ RadioButtons
+ ToggleButtons
+ Select
+ SelectMultiple
+ SelectionSlider
+ SelectionRangeSlider

These widgets can be used to choose between multiple alternatives. The `SelectMultiple` widget allows multiple selections, whereas `Dropdown`, `RadioButtons`, `ToggleButtons`, `SelectionSlider` and `Select` only allow one selection. `SelectionRangeSlider` returns exactly two selections.

Every widget, except the sliders, can return `Nothing` as it selected index.

In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE FlexibleContexts #-}
import IHaskell.Display.Widgets

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [2]:
-- Allows single selection
tgbs <- mkToggleButtons
dropdown <- mkDropdown
radio <- mkRadioButtons
select <- mkSelect
slider <- mkSelectionSlider

-- Allows multiple selections
msel <- mkSelectMultiple
rslider <- mkSelectionRangeSlider

### Single Selection Widgets

We can set the options with `Options $ OptionLabels [array]`. Let's see how can we select one of two functions with multiple selectors:

In [3]:
init w = do
    setField w Description "Function:"
    setField w Options (OptionLabels ["sin", "cos"])
    return w
    
init tgbs
init dropdown
init radio
init select
init slider

We can create a `SelectionHandler` function that will be run every time the value is changed. Let's synchronize them so all the selectors always display the same value.

In [4]:
setHandlerOpt w = setField w SelectionHandler $ do
    y <- getField w OptionalIndex
    case y of
        Just x -> do
            setField dropdown OptionalIndex $ Just x
            setField tgbs OptionalIndex $ Just x
            setField radio OptionalIndex $ Just x
            setField select OptionalIndex $ Just x
            setField slider Index x
        _ -> return ()
        
setHandler w = setField w SelectionHandler $ do
    x <- getField w Index
    setField dropdown OptionalIndex $ Just x
    setField tgbs OptionalIndex $ Just x
    setField radio OptionalIndex $ Just x
    setField select OptionalIndex $ Just x
    setField slider Index x
        

setHandlerOpt tgbs
setHandlerOpt dropdown
setHandlerOpt radio
setHandlerOpt select
setHandler slider

### Multiple Selection Widgets

In the multiple selection widget, you can select multiple items by doing Shift+Click. Let's do an example for selecting provinces (maybe for filtering later).

In [5]:
ccaa = 
    [ "Andalusia", "Aragon", "Asturias", "Balearic Islands", "Basque Country"
    , "Canary Islands", "Cantabria", "Castile and León", "Castilla-La Mancha", "Catalonia"
    , "Madrid", "Extremadura", "Galicia", "La Rioja", "Navarre", "Murcia", "Valencia"
    ]
    
setField msel Options (OptionLabels ccaa)
msel

We can also set the items selected with the `Indices` attribute (which is an Integer list).

In [9]:
setField msel Indices [0,2]
setField msel Indices [-1,3]

But maybe we to scroll a lot because we don't have enough space, which is a bit tiring. We can fix this setting the `Rows` attribute. If we set it to `Nothing`, we are letting the frontend decide the number of rows.

In [7]:
setField msel Rows $ Just 10

Now let's create a slider. This one also returns an array of `Indices`, but it's always of size 2.

In [7]:
setField rslider Options (OptionLabels ["Very bad", "Bad", "Regular", "Good", "Very good"])
rslider

As expected, we can get/set the selected values.

In [8]:
getField rslider Indices
setField rslider Indices [1,3]

[2,4]